In [27]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

In [41]:
path = 'data/processed'
benchmarks_df = pd.read_csv(os.path.join(path, 'a_benchmarks_2024-09-11.csv'))
schools_df = pd.read_csv(os.path.join(path, 'a_schools_2024-08-29.csv'))
courseSections_df = pd.read_csv(os.path.join(path, 'a_courseSections_2024-08-29.csv'))
courseSectionRosters_df = pd.read_csv(os.path.join(path, 'a_courseSectionRosters_2024-08-29.csv'))
scores_df = pd.read_csv(os.path.join(path, 'a_scores_2024-09-11.csv'))
vendorUsage_df = pd.read_csv(os.path.join(path, 'a_vendorUsage_2024-09-11.csv'))

/var/folders/w3/z8qntdcx62v2qmj6z9fwfhrh0000gp/T/ipykernel_44716/4073666699.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  courseSectionRosters_df = pd.read_csv(os.path.join(path, 'a_courseSectionRosters_2024-08-29.csv'))


# data exploration

## benchmarks

In [31]:
print("benchmarks_df")
print(benchmarks_df.shape)
print(benchmarks_df.columns)

benchmarks_df
(42, 5)
Index(['districtId', 'year', 'level', 'dataSource', 'value'], dtype='object')


In [32]:
benchmarks_df['year'] = pd.to_numeric(benchmarks_df['year'], errors='coerce').astype('int64')
benchmarks_df['level'] = pd.to_numeric(benchmarks_df['level'], errors='coerce').astype('int64')
benchmarks_df['value'] = pd.to_numeric(benchmarks_df['value'], errors='coerce').astype('float64')

print(benchmarks_df.dtypes)

districtId      int64
year            int64
level           int64
dataSource     object
value         float64
dtype: object


In [13]:
benchmarks_df

,districtId,year,level,dataSource,value
0,45,2017,3,value_scantronMath,2356.0
1,45,2017,3,value_scantronReading,2563.0
2,45,2017,4,value_scantronMath,2493.0
3,45,2017,4,value_scantronReading,2674.0
4,45,2017,5,value_scantronMath,2584.0
5,45,2017,5,value_scantronReading,2841.0
6,45,2017,6,value_scantronMath,2675.0
7,45,2017,6,value_scantronReading,2897.0
8,45,2017,7,value_scantronMath,2766.0
9,45,2017,7,value_scantronReading,3008.0


## schools

In [33]:
print("schools_df")
print(schools_df.shape)
print(schools_df.columns)

schools_df
(20, 8)
Index(['id', 'name', 'districtId', 'costCenter', 'abbreviation',
       'acqRequestSelectable', 'oneRoster_sourcedId', 'oneRoster_ticketId'],
      dtype='object')


In [35]:
schools_df.dtypes

id                        int64
name                     object
districtId                int64
costCenter                int64
abbreviation             object
acqRequestSelectable      int64
oneRoster_sourcedId     float64
oneRoster_ticketId        int64
dtype: object

In [15]:
schools_df

,id,name,districtId,costCenter,abbreviation,acqRequestSelectable,oneRoster_sourcedId,oneRoster_ticketId
0,222,Elementary A,45,5,EES,1,,0
1,223,Middle School A,45,8,EMS,1,,0
2,224,High School A,45,10,ECHS,1,,0
3,225,Junior High B,45,12,ECJH,1,,0
4,226,Elementary School B,45,18,HES,1,,0
5,227,Middle School C,45,19,HMS,1,,0
6,228,High School B,45,20,HHS,1,,0
7,229,Middle School D,45,23,MJH,1,,0
8,230,Elementary School C,45,30,RSE,1,,0
9,231,Elementary School D,45,35,CES,1,,0


## courseSections

In [36]:
print("courseSections_df")
print(courseSections_df.shape)
print(courseSections_df.columns)

courseSections_df
(17662, 4)
Index(['id', 'vanityId', 'year', 'schoolId'], dtype='object')


In [37]:
courseSections_df.dtypes

id           int64
vanityId    object
year         int64
schoolId     int64
dtype: object

In [17]:
courseSections_df.head()

,id,vanityId,year,schoolId
0,40388,PE 4,2015,222
1,40389,READING 4,2015,222
2,40390,HmroomPK6,2015,222
3,40391,LANGUAGE 4,2015,222
4,40392,SCIENCE 4,2015,222


## courseSectionRosters

In [48]:
print("courseSectionRosters_df")
print(courseSectionRosters_df.shape)
print(courseSectionRosters_df.columns)

courseSectionRosters_df
(612241, 5)
Index(['id', 'courseSectionId', 'studentId', 'studentLevel', 'ticket'], dtype='object')


In [39]:
courseSectionRosters_df.dtypes

id                  int64
courseSectionId     int64
studentId           int64
studentLevel        int64
ticket             object
dtype: object

In [49]:
courseSectionRosters_df.head()

,id,courseSectionId,studentId,studentLevel,ticket
0,1047696,39975,45333,4,
1,1047704,39975,56116,4,
2,1047706,39975,45345,4,
3,1047715,39975,45400,4,
4,1047729,39975,45413,4,


In [50]:
courseSectionRosters_df = pd.merge(courseSectionRosters_df, courseSections_df, left_on='courseSectionId', right_on='id', how='left', suffixes=('', ''))
courseSectionRosters_df = courseSectionRosters_df.drop(columns=['id_y'])
courseSectionRosters_df.head()

,id_x,courseSectionId,studentId,studentLevel,ticket,vanityId,year,schoolId
0,1047696,39975,45333,4,,MathGrade 4,2015,232
1,1047704,39975,56116,4,,MathGrade 4,2015,232
2,1047706,39975,45345,4,,MathGrade 4,2015,232
3,1047715,39975,45400,4,,MathGrade 4,2015,232
4,1047729,39975,45413,4,,MathGrade 4,2015,232


In [62]:
courseSectionRosters_df['year'].value_counts()

year
2015    228471
2017    129612
2018    129147
2016    125011
Name: count, dtype: int64

In [63]:
courseSectionRosters_df['studentLevel'].value_counts()

studentLevel
10    54246
9     52595
7     51315
1     49253
4     49171
8     48846
2     48820
12    47993
3     47529
11    47342
5     41209
6     40529
0     28458
99     2930
98     1290
97      670
96       45
Name: count, dtype: int64

In [57]:
list(courseSectionRosters_df['vanityId'].unique())

['MathGrade 4',
 'MathGrade 3',
 'SOC ST 4',
 'SCIENCE 4',
 'PE 4',
 'READING 4',
 'LANGUAGE 4',
 'SCIENCE 3',
 'SOC ST 3',
 'PE 3',
 'READING 3',
 'LANGUAGE 3',
 'AAS:Read-4',
 'AAS:Math-4',
 'AAS:Math-3',
 'AAS:Read-3',
 'PE 2',
 'SOC ST 2',
 'HOMEROOM',
 'SCIENCE 2',
 'LANGUAGE 2',
 'MATH 2',
 'READING 2',
 'PE 1',
 'READING 1',
 'SCIENCE 1',
 'LANGUAGE 1',
 'MATH 1',
 'SOC SCI 1',
 'READING K',
 'PE K',
 'HmroomPK6',
 'MATH 4',
 'MATH 3',
 'SOC ST 1',
 'Algebra IH',
 'MathGrade 5',
 'STUDY HALL',
 'Eng 8 H/Adv',
 'Advisory 8',
 'CONTISS 7-8',
 'WLDHIS H/Adv',
 'CareerPrep-A',
 'PhySci8H/Adv',
 'PE 8',
 'PhySci Gr 8',
 'MATH8 PREALG',
 'English Gr 8',
 'WLDHIS 1500',
 'ContIssu 7-8',
 'StuAide7-8NC',
 'English LA 5',
 'PE 5',
 'Science Gr 5',
 'SocialStuGr5',
 'Math6H/Adv',
 'SciGr6H/A',
 'ELAGrPK6H/Ad',
 'SoSt6H/A',
 'PE 6',
 'MATHINVET7-8',
 'MATH 7',
 'LANG ARTS 7',
 'PE 7',
 'SCIENCE LI 7',
 'Geog Gr7',
 'CivicsGr 7',
 'Advisory 7',
 'EnricMath78',
 'LifeSciH Gr7',
 'GeogH/AdvGr

In [59]:
# Group by vanityId (course name) and courseSectionId, then count students
course_sizes = courseSectionRosters_df.groupby(['vanityId', 'courseSectionId'])['studentId'].count().reset_index()

# Calculate average size for each course
average_course_sizes = course_sizes.groupby('vanityId')['studentId'].mean().sort_values(ascending=False)

# Display the results
print("Average course sizes:")
print(average_course_sizes)

Average course sizes:
vanityId
District Wide - IS     26829.750000
District Wide - ISS    26829.750000
District Wide - GA     11543.333333
Schoolwide -237sID      1490.750000
Schoolwide -229sID      1231.000000
Schoolwide -236sID      1182.000000
Schoolwide -233sID      1140.000000
Schoolwide -235sID       992.250000
Schoolwide -231sID       970.250000
Schoolwide -234sID       925.500000
Schoolwide -226sID       704.000000
Schoolwide -227sID       678.000000
Schoolwide -232sID       654.875000
Schoolwide -222sID       559.250000
Schoolwide -228sID       484.333333
Schoolwide -224sID       483.000000
Music Grade2             321.000000
Tech Ed Gr 2             321.000000
Tech Ed Gr 1             296.000000
Music Grade1             296.000000
Music GradeK             291.000000
Tech Ed Gr K             291.000000
ACT ROI                  207.250000
PE 1                     125.071429
PE 2                     124.571429
PE K                     121.785714
RTI Math Tier II         107.0000

In [51]:
# print number of times each student appears
student_counts = scores_df['studentId'].value_counts()
print(student_counts)
print(student_counts.describe())

studentId
48102    6
53036    6
48763    6
51819    6
48747    6
        ..
54623    1
54622    1
54621    1
54620    1
54521    1
Name: count, Length: 12211, dtype: int64
count    12211.000000
mean         2.810335
std          1.724629
min          1.000000
25%          1.000000
50%          3.000000
75%          5.000000
max          6.000000
Name: count, dtype: float64


## scores

In [40]:
scores_df.dtypes

studentId                             int64
studentLevel                          int64
year                                  int64
measurementTypeId                     int64
subgroup_specialEd                    int64
subgroup_lunchStatus                  int64
subgroup_gender                       int64
subgroup_ethnicity                    int64
subgroup_ell                          int64
subgroup_bottom25                     int64
value_actMathScore                  float64
value_actEnglishScore               float64
value_actReadingScore               float64
value_actScienceScore               float64
value_actCompositeScore             float64
value_nweaMath                      float64
value_nweaReading                   float64
value_nweaEnglish                   float64
value_iReadyMath                      int64
value_iReadyReading                   int64
value_actMasteryComposite           float64
value_actMasteryReading             float64
value_actMasteryMath            

In [18]:
scores_df.head()

,studentId,studentLevel,year,measurementTypeId,subgroup_specialEd,subgroup_lunchStatus,subgroup_gender,subgroup_ethnicity,subgroup_ell,subgroup_bottom25,value_actMathScore,value_actEnglishScore,value_actReadingScore,value_actScienceScore,value_actCompositeScore,value_nweaMath,value_nweaReading,value_nweaEnglish,value_iReadyMath,value_iReadyReading,value_actMasteryComposite,value_actMasteryReading,value_actMasteryMath,value_actMasteryScience,value_actMasteryEnglish,value_nweaELA,value_scantronMath,value_scantronReading,value_mapMath,value_mapEnglish,value_mapReading,value_mapRLA,value_scantronScience,value_scantronLanguage,value_ReadingFoundations,value_scantronAlgebra,value_eocMath,value_actELAScore,value_mapScience,value_LeapELAScale,value_LeapMathScale,value_LeapALScale,value_LeapE2Scale,value_LeapGmScale,value_LeapBlScale,value_LeapE3Scale,value_LeapUsScale,value_E1Scale,value_UsEsScale,value_accessReadingScale,value_accessReadingProficiency,value_accessCompositeScale,value_accesscompositeProficiency,value_dibelsReading,value_mapHistory,value_LEAPSocSci,value_scantronGeometry,value_MathMilestones,value_ELAMilestones,value_starReading,value_starMath,value_ScienceMilestones,value_LEAPScience,value_SocialMilestones,value_gcaInfo,value_gcaNar,value_gcaOpin,value_aimsWebMath,value_aimsWebReading
0,44774,9,2018,6,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,44774,8,2017,6,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2841.0,2950,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,44774,8,2017,4,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2808.0,2936,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,44775,8,2017,4,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2580.0,2660,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,44775,8,2017,6,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2705.0,2670,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
pd.set_option('display.max_columns', None)

In [16]:
print('scores_df')
print(scores_df.shape)
print(scores_df.columns)

scores_df
(303750, 69)
Index(['studentId', 'studentLevel', 'year', 'measurementTypeId',
       'subgroup_specialEd', 'subgroup_lunchStatus', 'subgroup_gender',
       'subgroup_ethnicity', 'subgroup_ell', 'subgroup_bottom25',
       'value_actMathScore', 'value_actEnglishScore', 'value_actReadingScore',
       'value_actScienceScore', 'value_actCompositeScore', 'value_nweaMath',
       'value_nweaReading', 'value_nweaEnglish', 'value_iReadyMath',
       'value_iReadyReading', 'value_actMasteryComposite',
       'value_actMasteryReading', 'value_actMasteryMath',
       'value_actMasteryScience', 'value_actMasteryEnglish', 'value_nweaELA',
       'value_scantronMath', 'value_scantronReading', 'value_mapMath',
       'value_mapEnglish', 'value_mapReading', 'value_mapRLA',
       'value_scantronScience', 'value_scantronLanguage',
       'value_ReadingFoundations', 'value_scantronAlgebra', 'value_eocMath',
       'value_actELAScore', 'value_mapScience', 'value_LeapELAScale',
       'value_L

In [19]:
# convert dtype to numeric
int_columns = ['studentId', 'studentLevel', 'year', 'measurementTypeId'] + [col for col in scores_df.columns if col.startswith('subgroup_')]
for col in int_columns:
    scores_df[col] = pd.to_numeric(scores_df[col], errors='coerce').astype('int64')

value_columns = [col for col in scores_df.columns if col.startswith('value_')]
for col in value_columns:
    scores_df[col] = pd.to_numeric(scores_df[col], errors='coerce').astype('float64')

scores_df.dtypes

studentId                             int64
studentLevel                          int64
year                                  int64
measurementTypeId                     int64
subgroup_specialEd                    int64
subgroup_lunchStatus                  int64
subgroup_gender                       int64
subgroup_ethnicity                    int64
subgroup_ell                          int64
subgroup_bottom25                     int64
value_actMathScore                  float64
value_actEnglishScore               float64
value_actReadingScore               float64
value_actScienceScore               float64
value_actCompositeScore             float64
value_nweaMath                      float64
value_nweaReading                   float64
value_nweaEnglish                   float64
value_iReadyMath                    float64
value_iReadyReading                 float64
value_actMasteryComposite           float64
value_actMasteryReading             float64
value_actMasteryMath            

In [20]:
print(f"specialEd {scores_df['subgroup_specialEd'].nunique()}")
print(f"lunchStatus {scores_df['subgroup_lunchStatus'].nunique()}")
print(f"gender {scores_df['subgroup_gender'].nunique()}")
print(f"ethnicity {scores_df['subgroup_ethnicity'].nunique()}")
print(f"ell {scores_df['subgroup_ell'].nunique()}")
print(f"bottom25 {scores_df['subgroup_bottom25'].nunique()}")


specialEd 6
lunchStatus 5
gender 5
ethnicity 5
ell 2
bottom25 2


In [21]:
scores_df['studentLevel'].value_counts()

studentLevel
 4     39257
 5     39053
 3     38960
 6     38309
 7     37406
 8     36716
 2     17089
 10    13941
 9     13388
 11    11835
 1      8208
 0      5333
 12     3254
 98      654
 99      275
 97       59
 96        9
-1         4
Name: count, dtype: int64

In [22]:
# remove columns that only contain zeros
scores_df = scores_df.loc[:, (scores_df != 0).any()]
scores_df.shape

(303750, 58)

In [23]:
scores_df.describe()

,studentId,studentLevel,year,measurementTypeId,subgroup_specialEd,subgroup_lunchStatus,subgroup_gender,subgroup_ethnicity,subgroup_ell,subgroup_bottom25,value_actMathScore,value_actEnglishScore,value_actReadingScore,value_actScienceScore,value_actCompositeScore,value_nweaMath,value_nweaReading,value_iReadyMath,value_iReadyReading,value_actMasteryComposite,value_actMasteryReading,value_actMasteryMath,value_actMasteryScience,value_actMasteryEnglish,value_nweaELA,value_scantronMath,value_scantronReading,value_mapMath,value_mapReading,value_mapRLA,value_scantronScience,value_scantronLanguage,value_ReadingFoundations,value_actELAScore,value_mapScience,value_LeapELAScale,value_LeapMathScale,value_LeapALScale,value_LeapE2Scale,value_LeapGmScale,value_LeapBlScale,value_LeapE3Scale,value_LeapUsScale,value_E1Scale,value_UsEsScale,value_accessReadingScale,value_accessReadingProficiency,value_accessCompositeScale,value_accesscompositeProficiency,value_dibelsReading,value_mapHistory,value_LEAPSocSci,value_MathMilestones,value_ELAMilestones,value_starReading,value_starMath,value_ScienceMilestones,value_SocialMilestones
count,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,253260.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000
mean,131440.302008,6.004998,2017.208234,2.757402,0.320174,0.256807,0.251562,0.793995,0.000040,0.016823,0.302838,0.297587,0.314677,0.310802,0.315253,16.406140,9.551638,25.415608,28.923687,0.014973,0.014347,0.015141,0.014779,0.015022,6.267480,1353.753873,1184.379088,24.613557,5.823842,18.694933,18.151335,22.339154,7.818683,0.008793,1.232902,107.269761,102.016665,7.949870,7.327658,7.663407,5.935779,6.001373,3.486887,6.552471,0.039272,0.518805,0.005376,0.510910,0.005271,1.627549,0.493528,58.640863,76.930199,72.340833,4.174281,86.041310,26.027691,26.343312
std,57982.885934,5.916790,10.375606,1.281897,6.673339,1.104788,1.134392,3.593934,0.010887,0.128608,2.334394,2.362907,2.483732,2.420651,2.418425,77.792264,43.052760,104.131783,120.020451,0.535933,0.522532,0.537850,0.530502,0.552539,63.973843,1247.204244,1346.178817,127.977722,61.696352,112.417341,215.191523,240.671880,139.455533,0.399529,24.358031,263.705178,256.420889,76.579856,73.894697,74.940800,64.739801,65.428454,50.162285,70.206616,5.107435,13.029753,0.144590,12.754386,0.134692,18.322074,17.891981,200.266543,184.741639,178.897702,48.159151,446.624951,113.742834,113.992499
min,44774.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,85484.000000,4.000000,2017.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000